In [1]:
import pandas as pd
import numpy as np

In [3]:
import torch
import sklearn
from sklearn.preprocessing import StandardScaler,LabelEncoder, MinMaxScaler

In [ ]:
#데이터 입력받기
xy_train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/train.csv")
X_test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/submit_sample.csv")# This Python 3 environment comes with many helpful analytics libraries installed

#테스트에서 존재하는 날짜가 전혀 달라 큰 의미가 없을것 같아 제했습니다.
#라벨값도 제외하여 x_train, y_train, x_test를 각기 구성했습니다.
X_train = xy_train.drop(['avgPrice', 'year'], axis=1)
X_test = X_test.drop('year', axis=1)
y_train = xy_train['avgPrice']

#torch를 받아왔습니다.
#옵티마이저와 선형모델, 목적함수를 불러오기 위한 optim 모듈과 nn모듈도 선언했습니다.
import torch
import torch.optim as optim
import torch.nn as nn

#각 데이터를 학습이 가능하도록 tensor의 형태로 바꾸어주엇습니다.
#이때 dataFrame은 들어가지 않아 values를 이용해서 해당하는 값만 들어갈 수 있도록 조정하였습니다.
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values).reshape(-1, 1)
#Linear를 설정할때, 가중치가 항상 동일하게 초기화될 수 있도록
#torch.manual_seed를 이용해서 랜덤시드를 고정시켜두었습니다.
torch.manual_seed(1)


#총 특징의 수가 4개이기에 이의 배수인 8로 피쳐를 넓힌 후 총 4개의 레이어로 구성하였습니다.
linear1 = nn.Linear(4, 8)
linear2 = nn.Linear(8, 8)
linear3 = nn.Linear(8, 8)
linear4 = nn.Linear(8, 1)

#활성화 함수로는 relu함수를 사용했습니다.
relu = nn.ReLU()

#만든 레이어와 활성함수를 sequential함수를 이용해서 하나로 묶어주었습니다.
model = nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4)

#옵티마이저는 adam을 사용했고 학습률로는 0.001을 설정했습니다.
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#평가기준이 mean squared error이기에 해당하는 목적함수를 사용했습니다.
loss = nn.MSELoss()
#총 1만번을 돌렸습니다.
#학습 후 cost를 구하고 cost의 변화량을 보기 위해서
#step이 1000마다 cost에 어떠한 변화가 있는지 각기 찍어보았습니다.

#그 후 10만번을 돌렸지만 유의미한 결과가 나오지는 않았습니다.

#
for step in range(10001):
    hypothesis = model(X_train)
    cost = loss(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%1000 == 0:
        print(f"step:{step}, cost:{cost.item()}")
        
        
#총 1만번을 돌렸을때 : 1565.91922
#총 10만번을 돌렸을때 : 
step:0, cost:12020879.0
step:1000, cost:2941765.75
step:2000, cost:1995167.25
step:3000, cost:1972851.375
step:4000, cost:1941357.875
step:5000, cost:1884304.75
step:6000, cost:1789588.75
step:7000, cost:1760711.375
step:8000, cost:1759755.375
step:9000, cost:1757862.375
step:10000, cost:1756609.875
#학습을 끝낸 후 model에 x_test를 넣어 예측값을 뽑은 후
#submit에 넣을 수 있게 np.array로 바꾸어주었습니다.

predict = np.array(model(X_test).detach())
submit['Expected'] = predict
submit.to_csv("1.csv", index=False)
결론

신경망으로 구성한 결과 처음 1만번을 돌렸을때도 baseline보다 높은 점수가 나오는 것을 확인할 수 있었습니다.

학습이 더 이루어질까 하여 10만번으로 수정해서 돌려본 결과 더 좋은 성능이 나오지는 않았습니다.

이를 고쳐보기 위해서 레이어를 조금 더 넓혀보았지만 큰 의미가 없었습니다. 그 후 층을 하나 줄여 은닉층을 하나만으로 구성하고 2만번을 돌려보았지만 기존과 비슷한 성능이 나와 처음대로 코드를 구성해두었습니다.

 
 
 
 